In [79]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import time
import random
import requests
from bs4 import BeautifulSoup
import nltk
import cltk

nltk.download('stopwords')
# Rends nos tracés inline
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 5)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\melassouri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [80]:
# dfTrustPilote=pd.read_csv('trustpilot.csv')
# dfTrustPilote

df=pd.read_csv('corpus.csv')
df

,date,user,rating,review
0,Commenté en France le 9 août 2018,Justine,1.0,Le téléphone n a jamais fonctionné et Apple no...
1,Commenté en France le 21 juin 2018,Gouyer Eric,1.0,Soit-disant le téléphone est certifié recondit...
2,Commenté en France le 13 décembre 2018,Peineau A.,5.0,"Reçu aujourd'hui via UPS standard, bien emball..."
3,Commenté en France le 23 juillet 2018,Solenn Laurent,1.0,"iPhone reçu, la batterie ne tient pas une jour..."
4,Commenté en France le 1 janvier 2019,val,1.0,J’ai offert ce portable à mon fils pour Noël.....
...,...,...,...,...
1342,Commenté en France le 16 août 2020,Kasei,3.0,"au bout d'une semaine, le téléphone m'indique ..."
1343,Commenté en France le 2 juillet 2020,Josué KAZI,4.0,Produit fiable.J’ai reçu le téléphone depuis u...
1344,Commenté en France le 21 avril 2020,Familyismyreligion,5.0,"Livraison très rapide, plus rapide que prévu e..."
1345,Commenté en France le 21 septembre 2019,charret,3.0,la batterie se décharge très vite .


In [81]:
print("1: {}, 2: {}, 3: {}, 4: {}, 5: {}".format(
len(df[df['rating'] == 1.0]),
len(df[df['rating'] == 2.0]),
len(df[df['rating'] == 3.0]),
len(df[df['rating'] == 4.0]),
len(df[df['rating'] == 5.0])))



1: 348, 2: 93, 3: 129, 4: 235, 5: 542


In [82]:
import re
import string
import spacy
import fr_core_news_md
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from cltk.lemmatize.french.lemma import LemmaReplacer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score


nlp = fr_core_news_md.load()
        
def pre_processing(review):
    #tokenization
    words = re.split(r'\W+', review) 

#     print(words)

    #lowerCase
    words = [word.lower() for word in words] 

    #suppression de la ponctuation
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in words]

    # suppression des mots non alpha
    words = [word for word in stripped if word.isalpha()]

    #suppression des stop words
    stop_words = stopwords.words('french')
    words = [w for w in words if not w in stop_words]
    return words

In [83]:
def stemmatization(words):
    stemmer = FrenchStemmer()
    stemmed = [stemmer.stem(word) for word in words]  
#     print(words)
#     print(stemmed)
    return stemmed

In [84]:
def lemmatization(words):
    words_string = stringify(words)
    spacy_words = nlp(words_string)
    lemmed = [word.lemma_ for word in spacy_words]
#     print(lemmed)
    return lemmed

In [85]:
def stringify(words):
    str = ""
    for word in words:
        str += word+" "
    return str


In [86]:
def wordsCountsIDF(words):
    for word in words:
        if word in words_countsIDF:
            words_countsIDF[word] += 1
        else:
            words_countsIDF[word] = 1
def wordsCountsTF(words):
    for word in words:
        if word in words_countsTF:
            words_countsTF[word] += 1
        else:
            words_countsTF[word] = 1 
    for word in words:
            words_countsTF[word]/len(words) 

In [87]:
def vectorise(words):
    vectorizer = TfidfVectorizer()
    vectorizer.fit(words)
    vector = vectorizer.transform(words)
    print(vector.shape)
    return vector

In [88]:
words_countsTF = {}     
words_countsIDF = {}    

TF = []
IDF = []
TFIDF = []

processed_review = []

def main():
    for review in df['review']: 
        words = pre_processing(review)
#         words_stem = stemmatization(words)
        words_lem = lemmatization(words)
        processed_review.append(stringify(words_lem))
main()
df['processed_review'] = processed_review 
x = vectorise(df['processed_review'])

df.loc[(df['rating'] > 3.0)]=1
df.loc[(df['rating'] < 3.0)]=0
y = df['rating']

['Le', 'téléphone', 'n', 'a', 'jamais', 'fonctionné', 'et', 'Apple', 'nous', 'l', 'a', 'signifié', 'volé', '']
['téléphone', 'avoir', 'jamais', 'fonctionner', 'apple', 'avoir', 'signifier', 'voler']
['Soit', 'disant', 'le', 'téléphone', 'est', 'certifié', 'reconditionné', 'Pourtant', 'l', 'article', 'fait', 'état', 'd', 'une', 'défectuosité', 'évident', 'et', 'flagrante', 'Le', 'haut', 'parleur', 'du', 'haut', 'celui', 'd', 'écoute', 'à', 'l', 'oreille', 'ne', 'fonctionne', 'pas', 'du', 'tout', 'L', 'article', 'a', 'probablement', 'été', 'repris', 'sans', 'être', 'testé', 'du', 'tout', 'par', 'le', 'vendeur', '']
['dire', 'téléphone', 'certifier', 'reconditionner', 'pourtant', 'article', 'faire', 'état', 'défectuosité', 'évident', 'flagrant', 'haut', 'parleur', 'haut', 'celui', 'écoute', 'oreille', 'fonctionne', 'tout', 'article', 'avoir', 'probablement', 'reprendre', 'sans', 'être', 'tester', 'tout', 'vendeur']
['Reçu', 'aujourd', 'hui', 'via', 'UPS', 'standard', 'bien', 'emballé', 'l

['juste', 'scandaleuxl', 'téléphone', 'recevoir', 'déjà', 'problèmesla', 'connectiqu', 'fonctionner', 'donc', 'renvoyer', 'bout', 'jour', 'utilisation', 'mois', 'après', 're', 'recevoir', 'téléphone', 'deviner', 'quoi', 'fonctionne', 'toujours', 'ce', 'fois', 'batterie', 'complètement', 'mourir', 'également', 'micro', 'vouloir', 'dire', 'téléphone', 'lequel', 'pouvoir', 'téléphoner', 'encore', 'heure', 'actuel', 'train', 'batailler', 'vendeur', 'tiers', 'demande', 'remboursement', 'dur', 'depuis', 'mois', 'car', 'vouloir', 'opérer', 'tel', 'solution', 'vouloir', 'renvoyer', 'encore', 'fois', 're', 'réparerbref', 'catastrophe', 'conseil', 'passer', 'chemin', 'sinon', 'aller', 'avoir', 'très', 'mauvais', 'surprise']
['L', 'adaptateur', 'ou', 'le', 'port', 'audio', 'est', 'défectueux', 'Impossible', 'd', 'ecouter', 'La', 'musique', 'plus', 'de', '25', 'sec', 'sans', 'que', 'le', 'téléphone', 'notifie', 'cet', 'accessoire', 'n', 'est', 'pas', 'reconnu', 'par', 'l', 'iphone', '']
['adaptate

['impeccable', 'griffure', 'comme', 'magasinje', 'regrette', 'choix', 'av', 'faire', 'confiance', 'amazon']
['Il', 'a', 'été', 'livré', 'très', 'vite', 'et', 'semble', 'fonctionner', 'parfaitement', 'Extérieurement', 'il', 'est', 'parfait', 'Mais', 'l', 'appareil', 'photo', 'ne', 'peut', 'pas', 'fonctionner', 'en', 'silencieux', 'Après', 'passage', 'à', 'l', 'Apple', 'store', 'il', 'm', 'a', 'été', 'dit', 'que', 'cet', 'appareil', 'était', 'd', 'origine', 'coréenne', 'om', 'la', 'loi', 'ne', 'permet', 'pas', 'de', 'prendre', 'des', 'photos', 'en', 'silencieux', 'pour', 'éviter', 'le', 'harcèlement', 'cette', 'caractéristique', 'n', 'est', 'pas', 'précisée', 'quand', 'on', 'achète', 'l', 'appareil', 'et', 'ce', 'n', 'est', 'pas', 'normal', 'Je', 'retourne', 'cet', 'iphone', 'pourtant', 'en', 'parfait', 'état', '']
['avoir', 'livrer', 'très', 'vite', 'semble', 'fonctionner', 'parfaitement', 'extérieurement', 'parfait', 'appareil', 'photo', 'pouvoir', 'fonctionner', 'silencieux', 'après',

['fonctionne', 'bien', 'beaucoup', 'trop', 'surchauffe', 'quand', 'utilise', 'min', 'bouillir', 'donc', 'vraiment', 'top', 'prix']
['Je', 'suis', 'très', 'déçue', 'et', 'très', 'en', 'colère', 'le', 'téléphone', 'en', 'lui', 'même', 'est', 'bien', 'il', 'ne', 'présente', 'que', 'des', 'micros', 'rayures', 'en', 'revanche', 'la', 'batterie', 'est', 'morte', 'C', 'EST', 'HONTEUX', 'il', 'est', 'impossible', 'de', 'ne', 'pas', 'le', 'voir', 'au', 'reconditionnement', 'c', 'est', 'vraiment', 'de', 'l', 'arnaque', 'C', 'est', 'un', 'cadeau', 'de', 'noel', 'pour', 'ma', 'fille', 'le', '25', 'décembre', 'elle', 'ouvre', 'son', 'cadeau', 'le', 'téléphone', 'ne', 's', 'allume', 'pas', 'je', 'vous', 'dit', 'pas', 'sa', 'tête', 'la', 'pauvre', 'La', 'batterie', 'etait', 'à', '0', 'nous', 'faisons', 'une', 'recharge', 'et', 'la', 'on', 'se', 'rend', 'compte', 'que', 'la', 'batterie', 'ne', 'tient', 'pas', 'plus', 'd', '1h', 'Bon', 'je', 'trouve', 'cela', 'etrange', 'mais', 'comme', 'ma', 'fille', 

['iphone', 'avoir', 'prix', 'là', 'bout', 'moi', 'commencer', 'plus', 'tenir', 'charge', 'batterie', 'tenir', 'journée', 'plus', 'téléphone', 'éteindre', 'rallume', 'tout', 'seul', 'importe', 'quand', 'trouve', 'scandaleu', 'vende', 'reconditionner', 'avoir', 'prix', 'là', 'fonctionner', 'correctement', 'pourtant', 'premier', 'fois', 'achète', 'reconditionner', 'jamais', 'acheter', 'avoir', 'revendeur', 'arnaque', 'simple']
['j', 'avais', 'deja', 'un', 'iphone', '6s', 'et', 'un', 'iphone', 'SEquand', 'j', 'ai', 'acqui', 'ce', 'iphone', '7', 'je', 'me', 'suis', 'rendu', 'compte', 'que', 'la', 'qualité', 'de', 'la', 'dalle', 'etait', 'similaire', 'a', 'celle', 'd', 'un', 'iphone', 'SE', 'et', 'non', 'au', 'moins', 'a', 'celle', 'd', 'un', 'iPhone', '6spour', 'le', 'vendeur', 'cela', 'serais', 'normal', 'je', 'ne', 'suis', 'pas', 'd', 'accord', 'pour', 'moi', 'je', 'ne', 'peut', 'pas', 'pensé', 'que', 'la', 'qualité', 'de', 'l', 'écran', 'soit', 'moins', 'bonne', 'qu', 'un', 'iphone', '6S

...

['très', 'très', 'mâle', 'emballer', 'boîte', 'origin', 'pourquoi', 'très', 'grave', 'envoyer', 'petit', 'pochette', 'plastique', 'sans', 'papier', 'bulle', 'choquer', 'très', 'peur', 'voir', 'arriver', 'comme', 'avoir', 'air', 'très', 'bon', 'état']
['J', 'ai', 'eu', 'un', 'très', 'mauvaise', 'expérience', 'avec', 'ce', 'téléphone', 'j', 'ai', 'recu', 'l', 'article', 'vendredi', '29', '06', 'toute', 'de', 'suite', 'je', 'me', 'suis', 'rendue', 'compte', 'q', 'il', 'avait', 'des', 'soucis', 'avec', 'le', 'câble', 'pour', 'charger', 'pas', 'original', 'Apple', 'le', 'téléphone', 'ne', 'se', 'charge', 'pas', 'bien', 'j', 'ai', 'branché', 'l', 'appareil', 'a', '5h', 'la', 'matin', 'à', '8h30', 'il', 'etait', 'encore', 'au', '36', 'vers', '12h00', 'il', 'était', 'chargé', '100', 'a', '15h30', 'j', 'avait', 'déjà', 'environ', '40', 'de', 'batterie', 'a', '18h00', 'j', 'avait', 'moins', 'de', '20', 'Le', 'telephone', 'n', 'est', 'pas', 'de', 'toute', 'nouveau', 'la', 'batterie', 'est', 'tota

['acheter', 'téléphone', 'premier', 'fois', 'renvoyer', 'problème', 'batterie', 'avoir', 'rembourser', 'reprendre', 'bien', 'éviter', 'vendeur', 'aujourd', 'hui', 'problème', 'précédent', 'sauf', 'là', 'carrément', 'rallume', 'plus', 'amener', 'chez', 'réparateur', 'car', 'sous', 'garantie', 'perdre', 'tout', 'si', 'renvoyer', 'fois', 'ouvert', 'réparateur', 'voir', 'batterie', 'avoir', 'changer', 'bien', 'origine', 'joint', 'étanchéité', 'remettre', 'donc', 'acheter', 'téléphone', 'étanche', 'pièce', 'qualité', 'médiocre', 'bravo', 'donc', 'payer', 'très', 'cher', 'produit', 'non', 'conforme', 'maintenant', 'devoir', 'payer', 'réparation', 'alors', 'ail', 'censé', 'être', 'sous', 'garantie', 'temps', 'compliqué', 'pouvoir', 'avoir', 'personne', 'téléphone', 'vraiment', 'pense', 'plus', 'racheter', 'amazon', 'produit', 'aussi', 'coûteux', 'si', 'solution', 'correct', 'apporter', 'vraiment', 'dommage']
['Le', 'téléphone', 'iPhone', '7', 'reconditionné', 'par', 'NEXT', 'MOBILES', 'foncti

['très', 'beau', 'téléphonecorrespond', 'tout', 'fait', 'descriptifsauf', 'batterieok', 'écrire', 'tout', 'iphones', 'vendre', 'verifier', 'etat', 'batterie', 'devoir', 'être', 'moins', 'apple', 'préconise', 'batterie', 'devoir', 'être', 'remplacédonc', 'peine', 'recevoir', 'falloir', 'remplacemerci', 'faire', 'effort', 'abuser', 'là']
['L', 'iphone', '7', 'est', 'petit', 'et', 'réactif', 'Par', 'contre', 'je', 'ne', 'met', 'qu', 'une', 'étoile', 'par', 'rapport', 'à', 'l', 'état', 'du', 'téléphone', 'que', 'j', 'ai', 'reçu', 'Celui', 'ci', 'était', 'tout', 'abîmé', 'marqués', 'de', 'chute', 'sur', 'le', 'coin', 'inférieur', 'gauche', 'batterie', 'ne', 'respectant', 'pas', 'les', 'critères', 'de', 'ventes', '']
['iphon', 'petit', 'réactif', 'contre', 'mettre', 'étoile', 'rapport', 'état', 'téléphone', 'recevoir', 'celui', 'ci', 'tout', 'abîmé', 'marqué', 'chute', 'coin', 'inférieur', 'gauche', 'batterie', 'respecter', 'critère', 'vente']
['Appareil', 'très', 'abîmé', 'batterie', 'qui',

['bonjour', 'attendre', 'toujours', 'facture', 'iphon', 'rose', 'sinon', 'rien', 'dire', 'très', 'content', 'produire']
['Reçu', 'très', 'rapidement', 'et', 'emballé', 'dans', 'le', 'respect', 'En', 'remplacement', 'du', 'même', 'mobile', 'Conforme', '']
['recevoir', 'très', 'rapidement', 'emballer', 'respect', 'remplacement', 'mobile', 'conforme']
['Téléphone', 'reconditionné', 'non', 'jap', 'quelques', 'légères', 'raillures', 'sur', 'le', 'flan', 'mais', 'rien', 'de', 'grave', 'La', 'batterie', 'n', 'a', 'pas', 'été', 'changé', 'mais', 'doit', 'être', 'à', '87', 'Bon', 'rapport', 'qualité', 'prix', 'pour', 'un', '128go']
['téléphone', 'reconditionner', 'non', 'jap', 'quelques', 'léger', 'raillure', 'flan', 'rien', 'grave', 'batterie', 'avoir', 'changer', 'devoir', 'être', 'bon', 'rapport', 'qualité', 'prix']
['Pour', 'ma', 'part', 'j', 'ai', 'rencontré', 'un', 'problème', 'technique', 'sur', 'ce', 'bel', 'IPhone', '7', 'Lors', 'des', 'appels', 'mes', 'correspondants', 'm', 'entendaie

['ecran', 'griffé']
['Super', 'produit', 'vraiment', 'nickel']
['super', 'produire', 'vraiment', 'nickel']
['Très', 'bonne', 'connaissance', 'du', 'produit']
['très', 'bon', 'connaissance', 'produire']
['téléphone', 'et', 'internet', '']
['téléphone', 'internet']
['Domaine', 'privé']
['domaine', 'priver']
['J', 'ai', 'bien', 'reçu', 'ma', 'mon', 'iPhone', 'c', 'est', 'je', 'suis', 'ravi', 'd', 'être', 'contente', 'merci', 'à', 'vous']
['bien', 'recevoir', 'iphone', 'ravir', 'être', 'content', 'merci']
['J', 'aime', 'beaucoup', 'le', 'dessin']
['aime', 'beaucoup', 'dessin']
['La', 'batrie', 'ce', 'decharge', 'trop', 'vite']
['batrie', 'decharge', 'trop', 'vite']
['trés', 'bien', 'comme', 'd', 'habitude']
['trés', 'bien', 'comme', 'habitude']
['il', 'marche', 'pas', 'bien']
['marche', 'bien']
['Je', 'suis', 'très', 'Satisfait', 'c', 'est', 'un', 'bon', 'produit', 'livrer', 'avant', 'la', 'date', 'prévue', 'le', 'seul', 'problème', 'ce', 'que', 'y', 'a', 'pas', 'les', 'écouteurs', 'mais',

In [72]:
predictors = x # df["vectorised_review"] 
target = y # df["rating"]

#On decoupe le dataset en 2 (test_size=0.2) soit 20% de test et 80% de train qui vont respectivement dans les val 
#_test _train
x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.2)

#On choisi un model pour predire notre resultat
cls=LogisticRegression(max_iter=200).fit(x_train,y_train)

#On calcul un score d'accurancy soit le nombre de prediction correcte obtenue
cls.score(x_val,y_val)

0.9037037037037037

In [73]:
cls=RandomForestClassifier(max_depth=12,n_estimators=300).fit(x_train,y_train)

cls.score(x_val,y_val)

0.9037037037037037

In [74]:
cls=GradientBoostingClassifier().fit(x_train,y_train)

cls.score(x_val,y_val)

0.9037037037037037